In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics.pairwise import rbf_kernel
from mliv.dgps import get_data, get_tau_fn, fn_dict
from mliv.rkhs import RKHSIV, RKHSIVCV, ApproxRKHSIV, ApproxRKHSIVCV

In [ ]:
def plot_est_vs_true(est, ind, T_test, T_train, true_fn, fn_name=None):
    plt.plot(T_test[:, ind], est.predict(T_test), label='est')
    plt.plot(T_test[:, ind], true_fn(T_test), '--', label='true')
    RMSE = np.sqrt(np.mean((est.predict(T_train).flatten() - true_fn(T_train).flatten())**2))
    R2 = 1 - RMSE**2 / np.var(true_fn(T_train).flatten())
    plt.title("RMSE on Train: {:.3f}, "
              "R2 on train: {:.2f}".format(RMSE, R2))
    plt.legend()
    if fn_name is not None:
        plt.savefig('{}_rkhs.png'.format(fn_name))
    plt.show()

# Data Generation

In [ ]:
n = 1000
n_z = 3
n_t = 3
iv_strength = .6
fname = 'abs'
dgp_num = 5
Z, T, Y, true_fn = get_data(n, n_z, iv_strength, get_tau_fn(fn_dict[fname]), dgp_num)

In [ ]:
ind = 0
x_grid = np.linspace(np.quantile(T[:, ind], .01), np.quantile(T[:, ind], .99), 100)
T_test = np.zeros((100, T.shape[1])) + np.median(T, axis=0, keepdims=True)
T_test[:, ind] = x_grid

In [ ]:
plt.figure(figsize=(10,3))
plt.subplot(1, 2, 1)
plt.scatter(Z[:, 0], Y)
plt.subplot(1, 2, 2)
plt.scatter(T[:, 0], Y)
plt.plot(T[np.argsort(T[:, 0]), 0], true_fn(T[np.argsort(T[:, 0])]))
plt.show()

# Visualization of Strength of Instrument Based on Kernel

In [ ]:
Kf = rbf_kernel(Z, gamma=2)
plt.imshow(Kf)
plt.show()

# Vanilla Benchmarks: OLS and 2SLS

In [ ]:
LinearRegression().fit(T, Y).coef_

In [ ]:
LinearRegression().fit(LinearRegression().fit(Z, T).predict(Z), Y).coef_

# RKHSIV 

In [ ]:
kernel = 'rbf'
delta_scale = 'auto'
delta_exp = .4
gamma = .1
alpha_scale = 'auto'
alpha_scales = np.geomspace(1, 10000, 10)
cv = 5

In [ ]:
est = RKHSIV(kernel=kernel, gamma=gamma, delta_scale=delta_scale,
             delta_exp=delta_exp, alpha_scale=alpha_scale).fit(Z, T, Y)

plot_est_vs_true(est, 0, T_test, T, true_fn)

### RKHSIV with CV Estimated Hyperparams

In [ ]:
est = RKHSIVCV(kernel=kernel, gamma=gamma, delta_scale=delta_scale,
               delta_exp=delta_exp, alpha_scales=alpha_scales, cv=cv).fit(Z, T, Y)

plot_est_vs_true(est, 0, T_test, T, true_fn)

In [ ]:
plt.title("Best alpha: {:.3f}".format(est.alpha_scales[np.argmin(est.avg_scores)]))
plt.scatter(est.alpha_scales, est.avg_scores)
plt.show()

### Oracle Hyperparameter Tuning

In [ ]:
scores = []
for t in alpha_scales:
    est = RKHSIV(kernel, gamma=gamma, delta_scale=delta_scale,
                 delta_exp=delta_exp, alpha_scale=t).fit(Z, T, Y)
    scores.append(np.sqrt(np.mean((est.predict(T) - true_fn(T))**2)))

In [ ]:
plt.title("Best alpha: {:.3f}".format(alpha_scales[np.argmin(scores)]))
plt.scatter(alpha_scales, scores)
plt.show()

In [ ]:
est = RKHSIV(kernel=kernel, gamma=gamma, delta_scale=delta_scale,
             delta_exp=delta_exp, alpha_scale=alpha_scales[np.argmin(scores)]).fit(Z, T, Y)

plot_est_vs_true(est, 0, T_test, T, true_fn)

# Nystrom Approx

If we assume that $K_z = U U'$ and $K_x= V V'$, then we first we observe that we can express the solution to the inner maximization problem as:
\begin{align}
\psi_n' U \left(\frac{1}{2n\delta^2}U'U + \frac{1}{2H}\right)^{-1} U'\psi_n
\end{align}
Let:
\begin{align}
Q = \left(\frac{1}{2n\delta^2}U'U + \frac{1}{2H} I\right)^{-1}
\end{align}
can show that we can express the solution in terms of $\gamma=V'a$ and such that $\gamma$ must solve:
\begin{align}
VV'\left(\left(U Q U' V + \lambda I) \gamma - UQU'y\right)\right) = 0
\end{align}
Or equivalently:
\begin{align}
V\left(\left(V'U Q U' V + \lambda I) \gamma - V'UQU'y\right)\right) = 0
\end{align}
Letting $A=V'U$, then we have that this is equivalent to:
\begin{align}
\gamma = (A Q A' + \lambda I)^{-1} A Q U'y
\end{align}
Then we can also solve for $a$ via: $V'a = \gamma \implies a = V^+\gamma$. 

However, typically we can express $h(x)$ as:
\begin{align}
h(x) = v_x' V'a = v_x' \gamma
\end{align}
where $v_x$ is a vector that corresponds to the feature map of a nystrom approximation for a target $x$, i.e.
\begin{align}
(K(x_1, x), \ldots, K(x_n, x)) =  V \phi(x)
\end{align}
such that $h(x) = \phi(x)' V' a = \phi(x)' \gamma$. All these calculations require time of the order of $n_\text{samples} \times n_\text{nystrom components}^2$

In [ ]:
kernel_approx = 'nystrom'
n_components = 500

In [ ]:
est = ApproxRKHSIV(kernel_approx=kernel_approx, n_components=n_components,
                    kernel=kernel, gamma=gamma, delta_scale=delta_scale,
                    delta_exp=delta_exp, alpha_scale=alpha_scale).fit(Z, T, Y)

plot_est_vs_true(est, 0, T_test, T, true_fn)

### Nystrom with CV Hyperparameter Tuning

In [ ]:
est = ApproxRKHSIVCV(kernel_approx=kernel_approx, n_components=n_components,
                      kernel=kernel, gamma=gamma, delta_scale=delta_scale,
                      delta_exp=delta_exp, alpha_scales=alpha_scales, cv=cv).fit(Z, T, Y)

plot_est_vs_true(est, 0, T_test, T, true_fn)

In [ ]:
plt.title("Best alpha scale: {:.3f}".format(est.alpha_scales[np.argmin(est.avg_scores)]))
plt.scatter(est.alpha_scales, est.avg_scores)
plt.show()

In [ ]:
est.best_alpha

### Random Fourier Features Approx

Same as Nystrom, but instead of choosing random points it chooses random fourier features to approximate the kernel

In [ ]:
kernel_approx = 'rbfsampler'
n_components = 200

In [ ]:
est = ApproxRKHSIV(kernel_approx=kernel_approx, n_components=n_components,
                    kernel=kernel, gamma=gamma, delta_scale=delta_scale,
                    delta_exp=delta_exp, alpha_scale=alpha_scale).fit(Z, T, Y)

plot_est_vs_true(est, 0, T_test, T, true_fn)

In [ ]:
est = ApproxRKHSIVCV(kernel_approx=kernel_approx, n_components=n_components,
                      kernel=kernel, gamma=gamma, delta_scale=delta_scale,
                      delta_exp=delta_exp, alpha_scales=alpha_scales, cv=cv).fit(Z, T, Y)

plot_est_vs_true(est, 0, T_test, T, true_fn)